In [3]:
cd GitHub/UPM-Master/Stock_Market_TS_Prediction


/Users/nicolacecere/GitHub/UPM-Master/Stock_Market_TS_Prediction


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

# from scripts.utils import add_seasonality
from sklearn.preprocessing import StandardScaler

2023-12-02 21:21:06.501220: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def create_train_label_split(data, label_column='Close'):
    """
    Splits a DataFrame into training data (X_train) and labels (y_label).
    X_train contains the first 80% of the rows, and y_label contains the last 20% of the specified label column.

    :param data: The input DataFrame.
    :param label_column: The name of the column to use for labels.
    :return: A tuple (X_train, y_label).
    """
    # Calculate the split index
    split_idx = int(len(data) * 0.8)

    # Split the data into training data and labels
    X_train = data.iloc[:split_idx, :].values
    y_label = data.iloc[split_idx:, :][label_column].values

    return X_train, y_label

In [5]:
def create_sequences_shift_t(data, t):
    X_train, y_train = [], []
    i = 0
    while i + 2 * t <= len(data):  # Ensure enough data for both X_train and y_train
        X_train.append(data.iloc[i : i + t, :].values)
        y_train.append(data.iloc[i + t : i + t + t]["Close"].values)
        i += t
    return np.array(X_train), np.array(y_train)

In [5]:
def create_sequences(data, t, y_step):
    X, y = [], []
    for i in range(len(data)-t-y_step-1):
        X.append(data.iloc[i:i+t, :].values)
        y.append(data.iloc[i + t: i+t+y_step, :]["Close"].values)
    return np.array(X), np.array(y)

In [6]:
df = pd.read_csv("data/sp500/csv/AAPL.csv")
df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")
df = df.sort_values(by="Date")

# Extract the year, month, and day as separate columns
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day"] = df["Date"].dt.day
df.drop(columns=["Date", "Adjusted Close"], inplace=True)

In [7]:
# Split the data into training and testing sets
train_size = int(len(df) * 0.8)
val_size = int(len(df) * 0.1)
train = df.iloc[:train_size, :]
val = df.iloc[train_size : train_size + val_size, :]
test = df.iloc[train_size + val_size :, :]

In [8]:
# Calculate the mean and standard deviation of each column in the training set
train_mean = train.mean()
train_std = train.std()

# Standardize the train, validation, and test sets using the calculated mean and std
train = (train - train_mean) / train_std
val = (val - train_mean) / train_std
test = (test - train_mean) / train_std

In [10]:
X_train, y_train = create_sequences(train, 22, 22)
X_val, y_val = create_sequences(val, 22, 22)
X_test, y_test = create_sequences(test, 22, 22)

In [12]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((8427, 22, 8),
 (8427, 22),
 (1014, 22, 8),
 (1014, 22),
 (1014, 22, 8),
 (1014, 22))

In [17]:
import tensorflow as tf
from tensorflow.keras import layers as tfkl
from tensorflow.keras.models import Model as tfkModel

def build_LSTM_regressor(input_shape, output_units):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # Feature extractor
    lstm = tfkl.LSTM(512, return_sequences=True)(input_layer)
    lstm = tfkl.LSTM(512)(lstm)
    dropout = tfkl.Dropout(.5)(lstm)

    # Output layer for regression
    output_layer = tfkl.Dense(output_units)(dropout)  # No activation for regression

    # Connect input and output through the Model class
    model = tfkModel(inputs=input_layer, outputs=output_layer, name='LSTM_regressor')

    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

    # Return the model
    return model

# Assuming input_shape is (t, number of features)
input_shape = (X_train.shape[1], X_train.shape[2])
output_units = y_train.shape[1]  # Number of days to predict

# Build and compile the model
model = build_LSTM_regressor(input_shape, output_units)
model.summary()

# Train the model
history = model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]
).history

Model: "LSTM_regressor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22, 8)]           0         
                                                                 
 lstm_2 (LSTM)               (None, 22, 512)           1067008   
                                                                 
 lstm_3 (LSTM)               (None, 512)               2099200   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 22)                11286     
                                                                 
Total params: 3177494 (12.12 MB)
Trainable params: 3177494 (12.12 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
2

KeyboardInterrupt: 

In [47]:
test_loss = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss (Mean Squared Error): {test_loss}")

InvalidArgumentError: Graph execution error:

Detected at node sub defined at (most recent call last):
  File "/Users/nicolacecere/opt/anaconda3/envs/DMTS/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/Users/nicolacecere/opt/anaconda3/envs/DMTS/lib/python3.9/runpy.py", line 87, in _run_code

  File "/Users/nicolacecere/opt/anaconda3/envs/DMTS/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/nicolacecere/opt/anaconda3/envs/DMTS/lib/python3.9/site-packages/traitlets/config/application.py", line 1077, in launch_instance

  File "/Users/nicolacecere/opt/anaconda3/envs/DMTS/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 737, in start

  File "/Users/nicolacecere/opt/anaconda3/envs/DMTS/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/Users/nicolacecere/opt/anaconda3/envs/DMTS/lib/python3.9/asyncio/base_events.py", line 601, in run_forever

  File "/Users/nicolacecere/opt/anaconda3/e

In [49]:
y_pred = model.predict(X_test)
# Plotting the actual vs predicted values
plt.figure(figsize=(12, 6))
plt.plot(y_test, label="Actual", color="blue", marker="o")
plt.plot(
    y_pred, label="Predicted", color="red", linestyle="dashed", marker="x"
)
plt.title("Actual vs Predicted Values")
plt.xlabel("Sample Index")
plt.ylabel("Value")
plt.grid(True)
plt.show()


ValueError: in user code:

    File "/Users/nicolacecere/opt/anaconda3/envs/DMTS/lib/python3.9/site-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/Users/nicolacecere/opt/anaconda3/envs/DMTS/lib/python3.9/site-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/nicolacecere/opt/anaconda3/envs/DMTS/lib/python3.9/site-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/nicolacecere/opt/anaconda3/envs/DMTS/lib/python3.9/site-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/Users/nicolacecere/opt/anaconda3/envs/DMTS/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   

In [48]:
X_train.shape

(5, 1260, 8)

In [33]:
y_pred

array([[1.4097782, 1.3396467, 1.2977468, ..., 1.4271364, 1.3305497,
        1.3861374],
       [1.4097782, 1.3396467, 1.2977468, ..., 1.4271364, 1.3305497,
        1.3861374],
       [1.4097782, 1.3396467, 1.2977468, ..., 1.4271364, 1.3305497,
        1.3861374],
       ...,
       [1.4097779, 1.3396468, 1.2977469, ..., 1.4271365, 1.3305497,
        1.3861375],
       [1.4097779, 1.3396468, 1.2977469, ..., 1.4271365, 1.3305497,
        1.3861375],
       [1.4097779, 1.3396468, 1.2977469, ..., 1.4271365, 1.3305497,
        1.3861375]], dtype=float32)

In [56]:
train.tail()

,Low,Open,Volume,High,Close,Year,Month,Day
9526,54.322498,55.195000,384986800,55.340000,54.415001,2018,9,21
9527,54.157501,54.205002,110773600,55.314999,55.197498,2018,9,24
9528,54.924999,54.937500,98217600,55.705002,55.547501,2018,9,25
9529,54.939999,55.250000,95938800,55.937500,55.105000,2018,9,26
9530,55.884998,55.955002,120724800,56.610001,56.237499,2018,9,27


In [57]:
test.tail()

,Low,Open,Volume,High,Close,Year,Month,Day
10585,141.919998,147.070007,64727200,147.300003,142.910004,2022,12,6
10586,140.000000,142.190002,69721100,143.369995,140.940002,2022,12,7
10587,141.100006,142.360001,62128300,143.520004,142.649994,2022,12,8
10588,140.899994,142.339996,76069500,145.570007,142.160004,2022,12,9
10589,141.070007,142.699997,20280660,143.000000,142.240005,2022,12,12


In [58]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1059 entries, 9531 to 10589
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Low     1059 non-null   float64
 1   Open    1059 non-null   float64
 2   Volume  1059 non-null   int64  
 3   High    1059 non-null   float64
 4   Close   1059 non-null   float64
 5   Year    1059 non-null   int32  
 6   Month   1059 non-null   int32  
 7   Day     1059 non-null   int32  
dtypes: float64(4), int32(3), int64(1)
memory usage: 53.9 KB
